In [10]:
import pandas as pd                                           
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse import spdiags
from scipy.stats import multivariate_normal
from copy import deepcopy
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import normalize

## Load Wikipedia data and extract TF-IDF features

In [11]:
wiki = pd.read_csv('people_wiki.csv')


As in the previous assignment, we extract the TF-IDF vector of each document. For your convenience, we extracted the TF-IDF vectors from the dataset. The vectors are packaged in a sparse matrix, where the i-th row gives the TF-IDF vectors for the i-th document. Each column corresponds to a unique word appearing in the dataset.


In [12]:
def load_sparse_csr(filename):
    loader = np.load(filename)
    data = loader['data']
    indices = loader['indices']
    indptr = loader['indptr']
    shape = loader['shape']
    
    return csr_matrix( (data, indices, indptr), shape)


tf_idf = load_sparse_csr('4_tf_idf.npz')
import json
with open('4_map_index_to_word.json', 'r') as f: # Reads the list of most frequent words
    map_index_to_word = json.load(f)

As in the previous assignment, we will normalize each document's TF-IDF vector to be a unit vector.



In [13]:
tf_idf = normalize(tf_idf)
tf_idf

<5000x100282 sparse matrix of type '<type 'numpy.float64'>'
	with 881415 stored elements in Compressed Sparse Row format>

### EM in high dimensions


EM for high-dimensional data requires some special treatment:

E step and M step must be vectorized as much as possible, as explicit loops are dreadfully slow in Python.
All operations must be cast in terms of sparse matrix operations, to take advantage of computational savings enabled by sparsity of data.
Initially, some words may be entirely absent from a cluster, causing the M step to produce zero mean and variance for those words. This means any data point with one of those words will have 0 probability of being assigned to that cluster since the cluster allows for no variability (0 variance) around that count being 0 (0 mean). Since there is a small chance for those words to later appear in the cluster, we instead assign a small positive variance (~1e-10). Doing so also prevents numerical overflow.
Due to complexity in implementation, we provide the complete implementation here. You are expected to answer some quiz questions using the results of clustering.

### Log probability function for diagonal covariance Gaussian.


In [14]:
def diag(array):
    n = len(array)
    return spdiags(array, 0, n, n)

def logpdf_diagonal_gaussian(x, mean, cov):
    '''
    Compute logpdf of a multivariate Gaussian distribution with diagonal covariance at a given point x.
    A multivariate Gaussian distribution with a diagonal covariance is equivalent
    to a collection of independent Gaussian random variables.

    x should be a sparse matrix. The logpdf will be computed for each row of x.
    mean and cov should be given as 1D numpy arrays
    mean[i] : mean of i-th variable
    cov[i] : variance of i-th variable'''

    n = x.shape[0]
    dim = x.shape[1]
    assert(dim == len(mean) and dim == len(cov))

    # multiply each i-th column of x by (1/(2*sigma_i)), where sigma_i is sqrt of variance of i-th variable.
    scaled_x = x.dot( diag(1./(2*np.sqrt(cov))) )
    # multiply each i-th entry of mean by (1/(2*sigma_i))
    scaled_mean = mean/(2*np.sqrt(cov))

    # sum of pairwise squared Eulidean distances gives SUM[(x_i - mean_i)^2/(2*sigma_i^2)]
    return -np.sum(np.log(np.sqrt(2*np.pi*cov))) - pairwise_distances(scaled_x, [scaled_mean], 'euclidean').flatten()**2

### EM algorithm for sparse data.



In [6]:
def log_sum_exp(x, axis):
    '''Compute the log of a sum of exponentials'''
    x_max = np.max(x, axis=axis)
    if axis == 1:
        return x_max + np.log( np.sum(np.exp(x-x_max[:,np.newaxis]), axis=1) )
    else:
        return x_max + np.log( np.sum(np.exp(x-x_max), axis=0) )

def EM_for_high_dimension(data, means, covs, weights, cov_smoothing=1e-5, maxiter=int(1e3), thresh=1e-4, verbose=False):
    # cov_smoothing: specifies the default variance assigned to absent features in a cluster.
    #                If we were to assign zero variances to absent features, we would be overconfient,
    #                as we hastily conclude that those featurese would NEVER appear in the cluster.
    #                We'd like to leave a little bit of possibility for absent features to show up later.
    n = data.shape[0]
    dim = data.shape[1]
    mu = deepcopy(means)
    Sigma = deepcopy(covs)
    K = len(mu)
    weights = np.array(weights)

    ll = None
    ll_trace = []

    for i in range(maxiter):
        # E-step: compute responsibilities
        logresp = np.zeros((n,K))
        for k in xrange(K):
            logresp[:,k] = np.log(weights[k]) + logpdf_diagonal_gaussian(data, mu[k], Sigma[k])
        ll_new = np.sum(log_sum_exp(logresp, axis=1))
        if verbose:
            print(ll_new)
        logresp -= np.vstack(log_sum_exp(logresp, axis=1))
        resp = np.exp(logresp)
        counts = np.sum(resp, axis=0)

        # M-step: update weights, means, covariances
        weights = counts / np.sum(counts)
        for k in range(K):
            mu[k] = (diag(resp[:,k]).dot(data)).sum(axis=0)/counts[k]
            mu[k] = mu[k].A1

            Sigma[k] = diag(resp[:,k]).dot( data.multiply(data)-2*data.dot(diag(mu[k])) ).sum(axis=0) \
                       + (mu[k]**2)*counts[k]
            Sigma[k] = Sigma[k].A1 / counts[k] + cov_smoothing*np.ones(dim)

        # check for convergence in log-likelihood
        ll_trace.append(ll_new)
        if ll is not None and (ll_new-ll) < thresh and ll_new > -np.inf:
            ll = ll_new
            break
        else:
            ll = ll_new

    out = {'weights':weights,'means':mu,'covs':Sigma,'loglik':ll_trace,'resp':resp}

    return out

In [15]:
from sklearn.cluster import KMeans

np.random.seed(5)
num_clusters = 25

# Use scikit-learn's k-means to simplify workflow

kmeans_model = KMeans(n_clusters=num_clusters, n_init=5, max_iter=400, random_state=1, n_jobs=1)
kmeans_model.fit(tf_idf)
centroids, cluster_assignment = kmeans_model.cluster_centers_, kmeans_model.labels_

means = [centroid for centroid in centroids]

In [16]:
num_docs = tf_idf.shape[0]
weights = []
for i in xrange(num_clusters):
    # Compute the number of data points assigned to cluster i:
    num_assigned = sum(cluster_assignment == i) # YOUR CODE HERE
    w = float(num_assigned) / num_docs
    weights.append(w)

In [17]:
covs = []
for i in xrange(num_clusters):
    member_rows = tf_idf[cluster_assignment==i]
    cov = (member_rows.multiply(member_rows) - 2*member_rows.dot(diag(means[i]))).sum(axis=0).A1 / member_rows.shape[0] \
          + means[i]**2
    cov[cov < 1e-8] = 1e-8
    covs.append(cov)

In [18]:
out = EM_for_high_dimension(tf_idf, means, covs, weights, cov_smoothing=1e-10)
print out['loglik'] # print history of log-likelihood over time

[3879297479.366981, 4883345753.533131, 4883345753.533131]


In [24]:
def visualize_EM_clusters(tf_idf, means, covs, map_index_to_word):
    print('')
    print('==========================================================')

    num_clusters = len(means)
    for c in xrange(num_clusters):
        print('Cluster {0:d}: Largest mean parameters in cluster '.format(c))
        print('\n{0: <12}{1: <12}{2: <12}'.format('Word', 'Mean', 'Variance'))
        
        # The k'th element of sorted_word_ids should be the index of the word 
        # that has the k'th-largest value in the cluster mean. Hint: Use np.argsort().
        sorted_word_ids = np.argsort(means[c])[::-1]  # YOUR CODE HERE

        for i in sorted_word_ids[:5]:
            print '{0: <12}{1:<10.2e}{2:10.2e}'.format({v:k for k, v in map_index_to_word.items()}[i], means[c][i], covs[c][i])
                                                       
        print '\n=====================================================Quiz Question. Select all the topics that have a cluster in the model created above. [multiple choice]===='

In [25]:
visualize_EM_clusters(tf_idf, out['means'], out['covs'], map_index_to_word)



Cluster 0: Largest mean parameters in cluster 

Word        Mean        Variance    
poetry      1.51e-01    1.90e-02
poems       6.33e-02    6.45e-03
poet        5.91e-02    6.36e-03
de          4.77e-02    8.72e-03
literary    4.68e-02    3.29e-03

=====================================================Quiz Question. Select all the topics that have a cluster in the model created above. [multiple choice]====
Cluster 1: Largest mean parameters in cluster 

Word        Mean        Variance    
she         1.60e-01    4.59e-03
her         1.04e-01    3.20e-03
music       1.53e-02    1.04e-03
actress     1.52e-02    1.14e-03
show        1.27e-02    7.33e-04

=====================================================Quiz Question. Select all the topics that have a cluster in the model created above. [multiple choice]====
Cluster 2: Largest mean parameters in cluster 

Word        Mean        Variance    
football    7.45e-02    4.57e-03
club        5.84e-02    2.55e-03
league      5.72e-02    2.

minister    1.28e-01    7.34e-03
prime       4.89e-02    4.79e-03
government  3.65e-02    2.00e-03
party       3.32e-02    1.33e-03
cabinet     3.25e-02    3.34e-03

=====================================================Quiz Question. Select all the topics that have a cluster in the model created above. [multiple choice]====
Cluster 21: Largest mean parameters in cluster 

Word        Mean        Variance    
news        5.64e-02    6.42e-03
radio       4.04e-02    3.61e-03
show        3.08e-02    2.28e-03
television  2.40e-02    1.07e-03
reporter    2.38e-02    1.88e-03

=====================================================Quiz Question. Select all the topics that have a cluster in the model created above. [multiple choice]====
Cluster 22: Largest mean parameters in cluster 

Word        Mean        Variance    
theatre     5.66e-02    7.14e-03
film        4.28e-02    1.66e-03
actor       3.92e-02    3.01e-03
television  3.67e-02    1.74e-03
comedy      3.52e-02    4.57e-03

==========

In [38]:
eje=np.array([1,7,12,56,2,98,4])
eje

array([ 1,  7, 12, 56,  2, 98,  4])

In [39]:
np.argsort(eje)[::-1]

array([5, 3, 2, 1, 6, 4, 0])